In [1]:
import os
import sys
from tqdm import tqdm
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import clear_output

sys.path.append('src')

from metrics import compute_metrics
from models import baseline_predict
%load_ext autoreload
%autoreload 2


In [2]:
DATA_PATH = 'lastfm-dataset-360K/'

In [3]:
train = pd.read_csv(DATA_PATH + 'train.csv')
valid = pd.read_csv(DATA_PATH + 'valid.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
lastfm_360_behav = pd.read_csv(DATA_PATH + 'behav-360k-processed.csv')
lastfm_360_demo = pd.read_csv(DATA_PATH + 'demo-360k-processed.csv')
lastfm_360_demo = lastfm_360_demo.set_index('user_email').astype(bool).reset_index()
test_users = np.load(DATA_PATH + 'test_users.npy')

In [4]:
artists = lastfm_360_behav['artist_id'].unique()
users = lastfm_360_demo['user_email'].unique()

In [5]:
k = 10

In [6]:
pred = baseline_predict(train)

In [7]:
pred_ratings = {u: np.array([pred.index.values, pred.values]) for u in test['user_email'].unique()}

In [8]:
_, _, _, _, _  = compute_metrics(test.drop(test[test.rating == 0].index), test['user_email'].unique(), pred_ratings, k)

Computing precision & recall...


  0%|          | 0/66883 [00:00<?, ?it/s]

Computing normalized discounted cumulative gain...


  0%|          | 0/66883 [00:00<?, ?it/s]

Computing hit rate...


  0%|          | 0/66883 [00:00<?, ?it/s]

Computing average reciprocal hit ranking...


  0%|          | 0/66883 [00:00<?, ?it/s]


Metrics: 

Precision @ 10: 0.01787748755289087
Recall    @ 10: 0.03605694219228302
Ndcg @ 10: 0.08824020983150574
Hit rate: 0.1781319617840109
Arhr: 0.06960940320089935


# Cold start

In [9]:
ratings = train.append(valid).append(test)
test_indices = ratings['user_email'].isin(test_users)
test_cold = ratings[ratings['user_email'].isin(test_users)]
train_cold = train[~train['user_email'].isin(test_users)]

In [10]:
pred = baseline_predict(train_cold)

In [11]:
pred_ratings = {u: np.array([pred.index.values, pred.values]) for u in test['user_email'].unique()}

In [12]:
_, _, _, _, _  = compute_metrics(test_cold.drop(test_cold[test_cold.rating == 0].index), test_cold['user_email'].unique(), pred_ratings, k)

Computing precision & recall...


  0%|          | 0/1000 [00:00<?, ?it/s]

Computing normalized discounted cumulative gain...


  0%|          | 0/1000 [00:00<?, ?it/s]

Computing hit rate...


  0%|          | 0/1000 [00:00<?, ?it/s]

Computing average reciprocal hit ranking...


  0%|          | 0/1000 [00:00<?, ?it/s]


Metrics: 

Precision @ 10: 0.17710000000000004
Recall    @ 10: 0.03317468900819118
Ndcg @ 10: 0.471630191175337
Hit rate: 1.763
Arhr: 0.7159726190476201
